In [9]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot = True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [30]:
import numpy as np 
image_data = np.load('training_np--30--30.npy')

# print len(image_data)
batch = mnist.train.next_batch(100)
print "batch data[0] is :", batch[0]

print "image data[0] is : ", image_data[0]

# todo save npy file as array[imagedata], array[labels]
# rather than array[imagedata, labels]
# perform batch jobs of ~20 or ~50 if more data available

batch data[0] is : [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
image data[0] is :  [ array([[127, 124, 126, 128, 127, 128, 128, 129, 129, 138, 153, 159, 156,
        141, 139, 141, 134, 128, 129, 128, 126, 128, 128, 128, 129, 128,
        128, 127, 127, 128],
       [127, 130, 128, 127, 127, 129, 129, 129, 134, 154, 148, 106, 101,
        114, 121, 133, 142, 133, 127, 128, 130, 128, 127, 129, 129, 128,
        127, 127, 128, 126],
       [127, 128, 128, 128, 130, 130, 127, 129, 143, 160, 111,  72,  87,
        105, 117, 124, 134, 133, 127, 127, 128, 131, 129, 128, 129, 128,
        128, 127, 130, 128],
       [130, 129, 128, 129, 129, 128, 129, 130, 146, 159,  91,  65,  92,
        121, 134, 133, 136, 134, 128, 128, 128, 127, 127, 127, 127, 129,
        127, 127, 129, 127],
       [128, 127, 126, 128, 128, 129, 129, 130, 139, 154, 1

In [11]:
import tensorflow as tf 

#makes tf more flexible about how we structure the code 
#allows to interleave oeprations which build a computation graph with ones that run the graph 
sess = tf.InteractiveSession()

#build a softmax regression model
# shape arg is optional, lets tf automatically catch bugs from inconsistent tensorshapes
x = tf.placeholder(tf.float32, shape=[None, 784]) # 28x28 pixel image = 784, None indicates frst dimension
y_ = tf.placeholder(tf.float32, shape = [None, 10]) # one hot 10-dimensional vector indictaing which digit class (0-9)

# Define weights and biases
W = tf.Variable(tf.zeros([784, 10])) # 784 input pixels by 10 output classes
b = tf.Variable(tf.zeros([10])) # 10 output classes



sess.run(tf.initialize_all_variables())

y = tf.matmul(x,W) + b # mutliply vectorizd input images, x, by the weight matrix W and add the bias, b

# calc loss function(indicates how bad the models prediction was on a single example, try to minimize accross all the examples)
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = y))


In [12]:
# lame nn with crappy mnist accuracy
#
## training (strightforward because tf knows the computation graph)

# train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# # apply gradient descent updates to the parameters
# for _ in range(1000):
#     batch = mnist.train.next_batch(100)
#     train_step.run(feed_dict={x: batch[0], y_: batch[1]})
    

# # evaluate the model
# correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

# # determining what fraction are correct, we cast to floats and then take the mean of the numbers (i.e. True becomes 1, False becomes 0)
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# # print accuracy 
# print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))



0.9162


In [13]:
#
###
###### MULTILAYER COMPUTATIONAL GRAPH

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

# add bias 0.1 to avoid 'dead neurons' from using ReLU neurons/ activation functions
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# convolution and pooling. handling the boundaries and stride sizes. This is the vanilla vesion (stride of 1, zero padding, output is same size as input)
# pooling is plain old max pooling over 2x2 blocks
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1],padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# first convolutional layer. consists of our first convolution, followed by max pooling
# convolution will compute 32 features for each 5x5 patch
# weight tensor wil thus have shape of [5, 5, 1, 32] i.e. [ patch size, patch size, input channels, output channels]
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32]) # bias vector

# reshape x to a 4d tensor, with the 2nd and 3rd dimensions corresponding to image width and height, last dimension is # of color channels
x_image = tf.reshape(x, [-1, 28, 28, 1])

# convolve x_image with the weight tesnor, add bias, apply ReLU function, then max pool
# max pool_2x2 resizes image size to 14x14
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# second convolutional layer 
# will have 64 features for each 5x5 patch
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# densely connected layer. Image size has been reduced to 7x7, we add a full_connected layer with 1024 neurons to allow processing on the entire image
# reshape the tensor from the pooling layer into a batch of vectors, multiple by a weight matrix, add bias, apply ReLU
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# add dropout to reduce overfitting - before readout layer. Dropout only really useful in large CNNs
# turn dropout on during training, turn it off during testing
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# readout layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2



In [27]:
# batch 50 images
batch = mnist.train.next_batch(50)

# 50 times 27*27 image flattened
print batch[0].shape

# 50 times 10-hot tensor label
print batch[1].shape




(50, 784)
(50, 10)


In [ ]:
# train and evalutate the model
# 


cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    for i in range(1000):
        #todo: collate image_data[0] in batches? or iterate through whole dict at once 
        batch = mnist.train.next_batch(50)
        if i % 50 == 0:
#             print batch[0][1]
#             print batch[1][1]
            
            print 
            train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
            
            print('step %d, training accuracy %g' % (i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
    
    prediction=tf.argmax(y,1)
    print "predictions", prediction.eval(feed_dict={x: mnist.test.images}, session=sess)


step 0, training accuracy 0.1

step 50, training accuracy 0.72

step 100, training accuracy 0.84

step 150, training accuracy 0.96
